In [222]:
import os
import re
import json
from pathlib import Path
from copy import deepcopy

In [223]:
BASE_ANNOTATION_DIR = Path("/media/grartem/B7DB5B36121B73AA/Projects/RL_robots/CommandClassifier/Data/Webanno/robot_commands_2022-11-14_1606/annotation/")

In [224]:
projects = {}
for projectDirName in ["2objects_crowd.txt", "3objects_crowd_1.txt", "3objects_crowd_2.txt",
                     "3objects_crowd_3.txt", "3objects_crowd_4.txt", "3objects_crowd_5.txt",
                    "direction_crowd.txt", "move_to_object_crowd.txt", "simple_crowd.txt"]:
    
    parsed_commands = {}
    for userFileName in ["student1.tsv", "student2.tsv", "PharmExpert_1.tsv"]:
        if not os.path.exists(BASE_ANNOTATION_DIR / projectDirName / userFileName):
            print("no", userFileName, "for", projectDirName)
            continue
        with open(BASE_ANNOTATION_DIR / projectDirName / userFileName, "r", encoding="utf-8") as f:
            parsed_commands[userFileName] = []
            skiping_attributes_line = False
            current_command = None
            for line in f:
                line = line.strip()
                if line=="":
                    continue
                if line[:6] == "#Text=":
                    line = line.encode().decode('unicode_escape')
                    if "Атрибуты: " in line:
                        # save old 
                        if current_command is not None:
                            parsed_commands[userFileName].append(current_command)
                        # begin new doc
                        current_command = {
                            "attributes_text": re.sub("#Text=Атрибуты: ", "", line),
                            "text": None,
                            "parse": []
                        }
                        skiping_attributes_line = True
                    elif "Текст: " in line:
                        current_command["text"] = re.sub("#Text=Текст: ", "", line)
                        skiping_attributes_line = False
                    else:
                        raise ValueError("???")
                elif line[0]=="#":
                    continue
                else:
                    if skiping_attributes_line:
                        continue
                    line = line.split("\t")
                    if len(line)==4:
                        if line[3]=="*":
                            print(projectDirName, word)
                            raise ValueError("Удали звёздочки в файлах, это косяки разметки")
                        # удаляем индексы сущностей, они нам вроде как не нужны 
                        line[3] = re.sub("\\[\d+\\]", "", line[3])
                    else:
                        line.append("_")
                    #current_command["parse"].append((line[2], line[3] if len(line)==4 else "_"))
                    current_command["parse"].append({"form":line[2],
                                                    "attribute_name":line[3]})
                    
            parsed_commands[userFileName].append(current_command)
    projects[projectDirName] = parsed_commands

no PharmExpert_1.tsv for 3objects_crowd_1.txt
no PharmExpert_1.tsv for 3objects_crowd_2.txt
no PharmExpert_1.tsv for 3objects_crowd_3.txt
no PharmExpert_1.tsv for move_to_object_crowd.txt


In [225]:
# проверка на то, что всегда есть только один разметчик
for projectDirName in ["2objects_crowd.txt", "3objects_crowd_1.txt", "3objects_crowd_2.txt",
                     "3objects_crowd_3.txt", "3objects_crowd_4.txt", "3objects_crowd_5.txt",
                    "direction_crowd.txt", "move_to_object_crowd.txt", "simple_crowd.txt"]:
    parsed_commands = projects[projectDirName]
    for i in range(len(parsed_commands['student1.tsv'])):
        if len(set([x["attribute_name"] for x in parsed_commands['student1.tsv'][i]['parse']])) > 1:
            assert len(set([x["attribute_name"] for x in parsed_commands['student2.tsv'][i]['parse']])) == 1 
            if 'PharmExpert_1.tsv' in parsed_commands:
                assert len(set([x["attribute_name"] for x in parsed_commands['PharmExpert_1.tsv'][i]['parse']])) == 1
        elif len(set([x["attribute_name"] for x in parsed_commands['student2.tsv'][i]['parse']])) > 1:
            assert len(set([x["attribute_name"] for x in parsed_commands['student1.tsv'][i]['parse']])) == 1 
            if 'PharmExpert_1.tsv' in parsed_commands:
                assert len(set([x["attribute_name"] for x in parsed_commands['PharmExpert_1.tsv'][i]['parse']])) == 1
        elif 'PharmExpert_1.tsv' in parsed_commands and len(set([x["attribute_name"] for x in parsed_commands['PharmExpert_1.tsv'][i]['parse']])) > 1:
            assert len(set([x["attribute_name"] for x in parsed_commands['student1.tsv'][i]['parse']])) == 1
            assert len(set([x["attribute_name"] for x in parsed_commands['student2.tsv'][i]['parse']])) == 1
        #print(parsed_commands['student1.tsv'][i]['parse'])
        #print(parsed_commands['student2.tsv'][i])
        #print(parsed_commands['PharmExpert_1.tsv'][i])

In [226]:
# проверка на то, что всегда есть хотя бы один разметчик
for projectDirName in ["2objects_crowd.txt", "3objects_crowd_1.txt", "3objects_crowd_2.txt",
                     "3objects_crowd_3.txt", "3objects_crowd_4.txt", "3objects_crowd_5.txt",
                    "direction_crowd.txt", "move_to_object_crowd.txt", "simple_crowd.txt"]:
    parsed_commands = projects[projectDirName]
    for i in range(len(parsed_commands['student1.tsv'])):
        if len(set([x["attribute_name"] for x in parsed_commands['student1.tsv'][i]['parse']])) > 1:
            continue
        elif len(set([x["attribute_name"] for x in parsed_commands['student2.tsv'][i]['parse']])) > 1:
            continue
        elif 'PharmExpert_1.tsv' in parsed_commands and len(set([x["attribute_name"] for x in parsed_commands['PharmExpert_1.tsv'][i]['parse']])) > 1:
            continue
        else:
            print("no annotation", projectDirName, i)
        #print(parsed_commands['student1.tsv'][i]['parse'])
        #print(parsed_commands['student2.tsv'][i])
        #print(parsed_commands['PharmExpert_1.tsv'][i])

no annotation 2objects_crowd.txt 6
no annotation 2objects_crowd.txt 71
no annotation 3objects_crowd_3.txt 155
no annotation 3objects_crowd_5.txt 167
no annotation move_to_object_crowd.txt 38


In [227]:
# оставить только одного разметчика
from collections import Counter
users_counter = Counter()
for projectDirName in ["2objects_crowd.txt", "3objects_crowd_1.txt", "3objects_crowd_2.txt",
                     "3objects_crowd_3.txt", "3objects_crowd_4.txt", "3objects_crowd_5.txt",
                    "direction_crowd.txt", "move_to_object_crowd.txt", "simple_crowd.txt"]:
    parsed_commands = projects[projectDirName]
    new_list = []
    for i in range(len(parsed_commands['student1.tsv'])):
        if len(set([x["attribute_name"] for x in parsed_commands['student1.tsv'][i]['parse']])) > 1:
            users_counter["student1"] += 1
            parse = parsed_commands['student1.tsv'][i]
        elif len(set([x["attribute_name"] for x in parsed_commands['student2.tsv'][i]['parse']])) > 1:
            users_counter["student2"] += 1
            parse = parsed_commands['student2.tsv'][i]
        elif 'PharmExpert_1.tsv' in parsed_commands and len(set([x["attribute_name"] for x in parsed_commands['PharmExpert_1.tsv'][i]['parse']])) > 1:
            users_counter["PharmExpert_1"] += 1
            parse = parsed_commands['PharmExpert_1.tsv'][i]
        else:
            users_counter["empty student1"] += 1
            # если нет, пофиг, берём первого и добавляем инфу оттуда
            parse = parsed_commands['student1.tsv'][i]
        simplified_command = deepcopy(parsed_commands['student1.tsv'][i])
        simplified_command["parse"] = simplified_command["parse"][2:]
        new_list.append(simplified_command)
    projects[projectDirName] = new_list

In [228]:
# проверить, нет ли каких-то корявых лейблов
labels_set = set()
for projectDirName in ["2objects_crowd.txt", "3objects_crowd_1.txt", "3objects_crowd_2.txt",
                     "3objects_crowd_3.txt", "3objects_crowd_4.txt", "3objects_crowd_5.txt",
                    "direction_crowd.txt", "move_to_object_crowd.txt", "simple_crowd.txt"]:
    parsed_commands = projects[projectDirName]
    for command in parsed_commands:
        for word in command["parse"]:
            labels_set.add(word["attribute_name"])
labels_set

{'_',
 'action',
 'direction',
 'object1',
 'object2',
 'object3',
 'relation1',
 'relation2'}

In [229]:
descriptions = {
    "action": ["патрулировать", "остановиться", "двигаться в направлении", "поворот в направлениие",
               "двигаться на заданное расстояние", "поворачивать на заданный угол", "двигаться к объекту",
               "искать", "обходить", "осматривать", "поворачивать к объекту", "прерваться", "продолжать", "анализ"],
    "direction": ["", "вперед", "назад", "направо", "налево", "на север", "на юг", "на восток", "на запад"],
    "meters": ["", "1", "3", "5", "7", "10", "15", "20", "25", "35", "50", "75","100"],
    "degshours": ["", "15", "30", "45", "60", "75","90", "105", "120", "135", "150","165", "180", "195", "210","225", "240", "255", "270","285", "300", "315", "330","345", "360"],
    "object1": ["ничего", "дом", "дерево", "сломанное дерево","лес", "яма", "человек", "холм",
                "трещина", "лежачий человек", "камень", "останец","барьер", "фонарь", "автомобиль", "circle", "route"],
    "nearest": ["",  "nearest_from_type"],
    "relation1": ["нет_отношения", "около", "позади", "перед","слева от", "правее"],
    "object2": ["ничего", "дом", "дерево", "сломанное дерево","лес", "яма", "человек", "холм",
                "трещина", "лежачий человек", "камень", "останец","барьер", "фонарь", "автомобиль", "circle", "route"],
    "relation2": ["нет_отношения", "около", "позади", "перед","слева от", "правее"],
    "object3": ["ничего", "дом", "дерево", "сломанное дерево","лес", "яма", "человек", "холм",
                "трещина", "лежачий человек", "камень", "останец","барьер", "фонарь", "автомобиль", "circle", "route"],
    "self": ["", "behind_of", "in_front_of", "to_left_from","to_right_from"],
    "gaze": ["", "has_gaze_focus_on"]
}
webannolabel_to_shortlabel = {
    '_': "O",
     'action': "A",
     'direction': "D",
     'object1': "O1",
     'object2': "O2",
     'object3': "O3",
     'relation1': "R1",
     'relation2': "R2"
}
for projectDirName in ["2objects_crowd.txt", "3objects_crowd_1.txt", "3objects_crowd_2.txt",
                     "3objects_crowd_3.txt", "3objects_crowd_4.txt", "3objects_crowd_5.txt",
                    "direction_crowd.txt", "move_to_object_crowd.txt", "simple_crowd.txt"]:
    parsed_commands = projects[projectDirName]
    for command in parsed_commands:
        attributes_vector = [0,0,0,0,0,0,0,0,0,0,0,0]
        for attr_pair in command["attributes_text"].split("; "):
            attr_name, attr_value = attr_pair.split("=")
            if attr_name=="действие":
                attributes_vector[0] = descriptions["action"].index(attr_value)
            elif attr_name=="объект№1":
                attributes_vector[4] = descriptions["object1"].index(attr_value)
            elif attr_name=="объект№2":
                attributes_vector[7] = descriptions["object2"].index(attr_value)
            elif attr_name=="объект№3":
                attributes_vector[9] = descriptions["object3"].index(attr_value)
            elif attr_name=="отношение о1-о2":
                attributes_vector[6] = descriptions["relation1"].index(attr_value)
            elif attr_name=="отношение о2-о3":
                attributes_vector[8] = descriptions["relation2"].index(attr_value)            
            elif attr_name=="отношение1-2":
                attributes_vector[6] = descriptions["relation1"].index(attr_value)
            elif attr_name=="отношение2-3":
                attributes_vector[8] = descriptions["relation2"].index(attr_value)
            elif attr_name=="направление":
                attributes_vector[1] = descriptions["direction"].index(attr_value)
            else:
                raise ValueError("Хз, что за атрибут")
        command["attributes_vector"] = attributes_vector
        for word in command["parse"]:
            word["attribute_name"] = webannolabel_to_shortlabel[word["attribute_name"]]
    with open("Data/ForWebannoProject/annotated/"+projectDirName.replace(".txt", ".jsonl"), "w") as f:
        for command in parsed_commands:
            f.write(json.dumps(command)+"\n")

In [238]:
# проверка лейблов
import pandas as pd
df = pd.read_csv("../Data/Raw/merged_with_labels_11_patterns.csv", sep=";")
df = df.loc[df["type"].isin(["students", "crowdsource"])]
df.drop_duplicates(inplace=True)

with open("Data/ForWebannoProject/annotated/simple_crowd.jsonl", "r") as f:
    lines = f.readlines()
idx = 0
for ir, row in df[df["y"]==0].iterrows():
    annotated = json.loads(lines[idx])
    assert annotated["attributes_vector"]==row[['action', 'direction', 'meters', 'degshours',
       'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3',
       'self', 'gaze']].tolist()
    idx += 1

with open("Data/ForWebannoProject/annotated/move_to_object_crowd.jsonl", "r") as f:
    lines = f.readlines()
idx = 0
for ir, row in df[df["y"]==5].iterrows():
    annotated = json.loads(lines[idx])
    assert annotated["attributes_vector"]==row[['action', 'direction', 'meters', 'degshours',
       'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3',
       'self', 'gaze']].tolist()
    idx += 1

with open("Data/ForWebannoProject/annotated/direction_crowd.jsonl", "r") as f:
    lines = f.readlines()
idx = 0
for ir, row in df[df["y"]==1].iterrows():
    annotated = json.loads(lines[idx])
    assert annotated["attributes_vector"]==row[['action', 'direction', 'meters', 'degshours',
       'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3',
       'self', 'gaze']].tolist()
    idx += 1

with open("Data/ForWebannoProject/annotated/2objects_crowd.jsonl", "r") as f:
    lines = f.readlines()
idx = 0
for ir, row in df[df["y"]==7].iterrows():
    annotated = json.loads(lines[idx])
    assert annotated["attributes_vector"]==row[['action', 'direction', 'meters', 'degshours',
       'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3',
       'self', 'gaze']].tolist()
    idx += 1

with open("Data/ForWebannoProject/annotated/3objects_crowd.jsonl", "r") as f:
    lines = f.readlines()
idx = 0
for ir, row in df[df["y"]==8].iterrows():
    annotated = json.loads(lines[idx])
    assert annotated["attributes_vector"]==row[['action', 'direction', 'meters', 'degshours',
       'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3',
       'self', 'gaze']].tolist()
    idx += 1


In [240]:
df[df["y"].isin([0,1,5,7,8])]

,x,y_name,y,action,direction,meters,degshours,object1,nearest,relation1,object2,relation2,object3,self,gaze,type
1,постереги,simple,0,0,0,0,0,0,0,0,0,0,0,0,0,students
2,дай по тормозам,simple,0,1,0,0,0,0,0,0,0,0,0,0,0,students
3,ваша остановка,simple,0,1,0,0,0,0,0,0,0,0,0,0,0,students
4,припаркуйся,simple,0,1,0,0,0,0,0,0,0,0,0,0,0,students
5,приехали,simple,0,1,0,0,0,0,0,0,0,0,0,0,0,students
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49457,"возле лесо стоит человек, за лесом есть повале...",relation2,8,6,0,0,0,3,0,2,4,1,6,0,0,crowdsource
49458,"Отправляйся к спиленному деревуза лесополосой,...",relation2,8,6,0,0,0,3,0,2,4,1,6,0,0,crowdsource
49459,"Шагай в сторону дерева, лежащего позади леса, ...",relation2,8,6,0,0,0,3,0,2,4,1,6,0,0,crowdsource
49460,"Рядом с человеком есть лес, за которым повален...",relation2,8,6,0,0,0,3,0,2,4,1,6,0,0,crowdsource
